In [3]:
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path
import gmaps
# Import API key
#from api_keys import geoapify_key

In [4]:
csv_file = Path(r"C:\cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,25.24,87,20,2.57,US,1715413994
1,1,ebaye,8.7748,167.7392,27.01,83,75,2.06,MH,1715413994
2,2,ushuaia,-54.8000,-68.3000,1.81,75,40,4.12,AR,1715413982
3,3,georgetown,5.4112,100.3354,31.96,74,20,4.12,MY,1715413995
4,4,stanley,54.8680,-1.6985,16.51,81,0,0.51,GB,1715413995


In [5]:
import warnings
warnings.filterwarnings("ignore")

# Configure the map plot
city_data_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", \
                                        frame_width=700, frame_height=500, \
                                        size="Humidity", scale=0.5, color="City")


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
temp_condition = ((city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 27))
wind_condition = (city_data_df["Wind Speed"] < 5)
#humid_condition = ((city_data_df["Humidity"] > 20) & (city_data_df["Humidity"] < 60))
#note - removing the above condition, since its resulting in a fewer data set.
cloud_condition = (city_data_df["Cloudiness"] < 10)

city_data_fil = city_data_df.loc[(temp_condition & wind_condition & cloud_condition ), :]

# Drop any rows with null values
city_final = city_data_fil.dropna()

# Display sample data
city_final

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,kaka,37.3482,59.6143,25.78,24,0,4.55,TM,1715413999
79,79,puerto real,36.5282,-6.1901,22.56,52,0,3.09,ES,1715414010
116,116,waddan,29.1614,16.1390,20.29,48,0,4.60,LY,1715414016
166,166,metlili chaamba,32.2667,3.6333,20.52,30,0,1.03,DZ,1715414028
182,182,colonia,50.9333,6.9500,20.70,65,0,1.03,DE,1715414011
183,183,al bawiti,28.3492,28.8659,26.49,27,0,4.63,EG,1715414031
196,196,belyy yar,53.6039,91.3903,23.94,41,0,1.00,RU,1715414034
215,215,quibala,-10.7337,14.9800,22.47,46,6,1.76,AO,1715414038
226,226,la passe,45.5549,-0.8967,20.57,73,0,4.01,FR,1715414040
267,267,awjilah,29.1081,21.2869,21.52,46,0,3.89,LY,1715414047


In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_final.copy()

hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
24,kaka,TM,37.3482,59.6143,24,
79,puerto real,ES,36.5282,-6.1901,52,
116,waddan,LY,29.1614,16.1390,48,
166,metlili chaamba,DZ,32.2667,3.6333,30,
182,colonia,DE,50.9333,6.9500,65,
183,al bawiti,EG,28.3492,28.8659,27,
196,belyy yar,RU,53.6039,91.3903,41,
215,quibala,AO,-10.7337,14.9800,46,
226,la passe,FR,45.5549,-0.8967,73,
267,awjilah,LY,29.1081,21.2869,46,


In [8]:
radius = 10000
limit = 5
apiKey = geoapify_key
params = {
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    params["categories"] = f'accommodation.hotel'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

NameError: name 'geoapify_key' is not defined